In [1]:
import os, sys

os.chdir("../../..")
print(os.path.abspath("./"))



/data/projects/src/github.com/any35/MOS


In [3]:
from run.pretrain.cls_mmae_sam.batch_soft_dice import DiceScore
from run.pretrain.cls_mmae_sam.model.sam_model_simple import SamModelSimple
from run.pretrain.text_mae_sam.model_factory import ModelFactory
from run.pretrain.text_mae_sam.sam_dataset_compat_simple import SAMDatasetCompatSimple
import torch
from torch import Tensor,autocast

device = "cuda:0"
run_name = 'text_mae_sam_simple2-reprod-0'
output_dir = f".checkpoint/{run_name}"
dataset = 'reprod'
valid_dataset_id = 0
eval_batch_size = 149

model_factory = ModelFactory(device, run_name)
model: SamModelSimple = model_factory.load_model(f"{output_dir}/latest")

new_dataset = SAMDatasetCompatSimple(
            base_path=f".cache/dataset/text-mae-sam-dataset/dataset-{dataset}-{valid_dataset_id}.pt",
            device=device,
            only_cmri_dataset=True,
        )
model.eval()

print("evaluating...")
dice_losses = []
avg_hausdorff_dist_losses = []
max_hausdorff_dist_losses = []
model.eval()
KEEP_WORST_COUNT = 30
last_image_list: list[tuple[Tensor, Tensor, Tensor]] = []
last_dice_list = []

calc_dice_score = DiceScore()

with torch.no_grad(), autocast(enabled=True):
    for batch in new_dataset.shuffer_valid_id(eval_batch_size):
        image, labeled_segment, token = new_dataset.batch_get_valid(batch)

        pred_masks = model.forward_image(image, token)

        # (bs, h, w)
        first_pred_masks = pred_masks[:, 0, 0, :, :]

        dice_loss = calc_dice_score(first_pred_masks, labeled_segment, reduce=False).tolist()
        dice_losses.extend(dice_loss)

        # 求最差的KEEP_WORST_COUNT个
        dice_loss = [(i, loss) for i, loss in enumerate(dice_loss)]
        dice_loss.sort(key=lambda x: x[1])
        dice_loss_index = [i for i, _ in dice_loss[:KEEP_WORST_COUNT]]
        last_image_list.append(
            (
                image[dice_loss_index],
                labeled_segment[dice_loss_index],
                first_pred_masks[dice_loss_index],
            )
        )
        dice_list = [score for _, score in dice_loss[:KEEP_WORST_COUNT]]
        last_dice_list.extend(dice_list)

        if len(last_image_list) > 1:  # 列表只保留最差的KEEP_WORST_COUNT个
            last_image_list = torch.cat(last_image_list, dim=0)
            last_dice_list = [(i, loss) for i, loss in enumerate(last_dice_list)]
            last_dice_list.sort(key=lambda x: x[1])
            dice_loss_index = [i for i, _ in last_dice_list[:KEEP_WORST_COUNT]]
            last_dice_list = [score for _, score in last_dice_list[:KEEP_WORST_COUNT]]
            last_image_list = [
                (
                    image[dice_loss_index],
                    labeled_segment[dice_loss_index],
                    first_pred_masks[dice_loss_index],
                )
            ]

# 只选dice取最差的KEEP_WORST_COUNT个
image, labeled_segment, first_pred_masks = last_image_list[0]


ImportError: cannot import name 'SamModel' from 'run.pretrain.cls_mmae_sam.model' (unknown location)